In [4]:
from pyspark.sql import SparkSession
# from datetime import datetime, date
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
import os
from numpy import nan
# from pyspark.sql import Row
# from pyspark.sql.functions import lit,regexp_replace, col, trim, regexp_extract
# spark = SparkSession.builder.appName("Masters").getOrCreate()
# spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
# pd.set_option('display.max_rows', 30)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', 10)
# pd.set_option('display.max_colwidth', None)




In [5]:
taxonomy_excel = pd.read_excel("/home/rszczygielski/bioinf/magisterka/geneBank/results/taxonomy1.xlsx")
array_of_values = taxonomy_excel.values
list_of_values = []
for row_array in array_of_values:
    row_array = row_array[~pd.isnull(row_array)]
    row_list = list(row_array)
    list_of_values.append(row_list)


taxonomy_row_list = []
for elem in list_of_values:
    new_elem = list(map(str.strip, elem))
    taxonomy_row_list.append(new_elem)
    # print(new_elem)
taxonomy_row_list[0]


["'Chordata'",
 "'Craniata'",
 "'Vertebrata'",
 "'Chondrichthyes'",
 "'Elasmobranchii'"]

In [6]:
df_masters = pd.read_csv("/home/rszczygielski/bioinf/magisterka/geneBank/results/Organisms_mitochondion_2.txt",\
    header=0,\
    delimiter='\t',\
    usecols=["ACCESSION", "ORGANISM","TAXONOMY", "PREVIOUS_GENE","CONTROL_REGION", "NEXT_GENE"],\
    index_col="ACCESSION")

df_taxonomy = df_masters[["TAXONOMY", "PREVIOUS_GENE","CONTROL_REGION", "NEXT_GENE"]]



In [7]:
def get_gene_info(aggragated_taxonomy, gene_column_name):
    if gene_column_name == "CONTROL_REGION":
        gen_info_list = aggragated_taxonomy[gene_column_name].unique().tolist()
    else:
        gen_info_list = aggragated_taxonomy[gene_column_name].unique()
    if len(gen_info_list) == 0:
        return ["Not found", "Not found"]
    gene_set = set()
    specific_gene_set = set()
    for gene in gen_info_list:
        if gene[-4:] == "None":
            splited_gene = gene.strip().split()
            gene_info = splited_gene[0]
            specific_gene_info = str(None)
        elif gene_column_name == "CONTROL_REGION":
            splited_gene = gene.strip().split()
            gene_info = splited_gene[0]
            specific_gene_info = splited_gene[-1]
        else:
            splited_gene = gene.strip().split("[")
            gene_info = splited_gene[0].strip()
            specific_gene_info = splited_gene[-1][:-1]
        if specific_gene_info == "region']":
            splited_gene = gene.strip().split("[")
            specific_gene_info = splited_gene[-1][:-1]
        gene_set.add(gene_info)
        specific_gene_set.add(specific_gene_info)
    joind_gene = ", ".join(list(gene_set))
    joind_specific_gene_set = ", ".join(list(specific_gene_set))
    return [joind_gene, joind_specific_gene_set]


In [8]:

general_table = pd.DataFrame(columns=["TAXNONOMY", "PREVIOUS_GENE_PRODUCT","PREVIOUS_GENE_SPECIFIC_PRODUCT",
                                "CONTROL_REGION_PRODUCT", "CONTROL_REGION_SPECIFIC_PRODUCT", "NEXT_GENE_PRODUCT", "NEXT_GENE_SPECIFIC_PRODUCT"])
for row_excel_taxonomy in enumerate(taxonomy_row_list):
    joined_values_entry = ", ".join(row_excel_taxonomy[1])
    aggragated_taxonomy = df_taxonomy.where(df_taxonomy["TAXONOMY"].str.contains(joined_values_entry)).dropna()
    # aggragated_taxonomy.to_excel(f"/home/rszczygielski/bioinf/magisterka/geneBank/results/aggregations/{joined_values_entry}.xlsx")
    previos_gene = get_gene_info(aggragated_taxonomy, "PREVIOUS_GENE")
    control_region = get_gene_info(aggragated_taxonomy, "CONTROL_REGION")
    next_gene = get_gene_info(aggragated_taxonomy, "NEXT_GENE")
    gene_info_df = pd.DataFrame(data=[[joined_values_entry, previos_gene[0], previos_gene[1], control_region[0], control_region[1], next_gene[0], next_gene[1]]],
                                columns=["TAXNONOMY", "PREVIOUS_GENE_PRODUCT","PREVIOUS_GENE_SPECIFIC_PRODUCT",
                                "CONTROL_REGION_PRODUCT", "CONTROL_REGION_SPECIFIC_PRODUCT", "NEXT_GENE_PRODUCT", "NEXT_GENE_SPECIFIC_PRODUCT"])
    general_table = general_table.append(gene_info_df)
    # gene_info_df.set_index("TAXNONOMY").to_excel(f"/home/rszczygielski/bioinf/magisterka/aggregations/gene_info/GENE_INFO_{joined_values_entry}.xlsx")
    # general_table[{"TAXNONOMY": joined_values_entry,"PREVIOUS_GENE_PRODUCT": previos_gene[0], "PREVIOUS_GENE_SPECIFIC_PRODUCT": previos_gene[1], "CONTROL_REGION_PRODUCT":
    #                control_region[0],"CONTROL_REGION_SPECIFIC_PRODUCT": control_region[1],"NEXT_GENE_PRODUCT": next_gene[0],"NEXT_GENE_SPECIFIC_PRODUCT": next_gene[1]}]

general_table.set_index("TAXNONOMY").to_excel(f"/home/rszczygielski/bioinf/magisterka/geneBank/results/GeneralGeneInfo.xlsx")
# \\wsl.localhost\Ubuntu-20.04\home\rszczygielski\bioinf\magisterka\aggregations\gene_info

/tmp/ipykernel_616/2317890857.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  general_table = general_table.append(gene_info_df)
/tmp/ipykernel_616/2317890857.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  general_table = general_table.append(gene_info_df)
/tmp/ipykernel_616/2317890857.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  general_table = general_table.append(gene_info_df)
/tmp/ipykernel_616/2317890857.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  general_table = general_table.append(gene_info_df)
/tmp/ipykernel_616/2317890857.py:13: FutureWarning: The frame.append method is deprecated and will be remove

In [9]:
# test = ", ".join(taxonomy_row_list[6])
# aggragated_taxonomy = df_taxonomy.where(df_taxonomy["TAXONOMY"].str.contains(test)).dropna()
# len(aggragated_taxonomy)
# gen_info_list = aggragated_taxonomy["PREVIOUS_GENE"].unique().tolist()
# splited_gen_info = np.char.split(gen_info_list, " ")
# gene_set = set()
# specific_gene_set = set()
# for gene in splited_gen_info:
#     print(gene)
#     gene_set.add(gene[8])
#     specific_gene_set.add(gene[-1])

# def get_gene_info(aggragated_taxonomy, gene_column_name):
#     gen_info_list = aggragated_taxonomy[gene_column_name].unique().tolist()
#     splited_gen_info = np.char.split(gen_info_list, " ")
#     gene_set = set()
#     specific_gene_set = set()
#     for gene in splited_gen_info:
#         gene_set.add(gene[8])
#         specific_gene_set.add(gene[-1])
#     return gene_set, specific_gene_set


# def get_gene_info(aggragated_taxonomy, gene_column_name):
#     if gene_column_name == "CONTROL_REGION":
#         gen_info_list = aggragated_taxonomy[gene_column_name].unique().tolist()
#         print(gen_info_list)
#     else:
#         gen_info_list = aggragated_taxonomy[gene_column_name].unique()
#     # print(gen_info_list.tolist())
#     if len(gen_info_list) == 0:
#         return ["Not found", "Not found"]
#     gene_set = set()
#     specific_gene_set = set()
#     for gene in gen_info_list:
#         if gene[-4:] == "None":
#             splited_gene = gene.strip().split()
#             gene_info = splited_gene[0]
#             specific_gene_info =  "None"
#         elif gene_column_name == "CONTROL_REGION":
#             splited_gene = gene.strip().split()
#             gene_info = splited_gene[0]
#             specific_gene_info = splited_gene[-1]
#         else:
#             splited_gene = gene.strip().split("[")
#             gene_info = splited_gene[0].strip()
#             specific_gene_info = splited_gene[-1][:-1]
#         if specific_gene_info == "region']":
#             splited_gene = gene.strip().split("[")
#             specific_gene_info = splited_gene[-1][:-1]
#         gene_set.add(gene_info)
#         specific_gene_set.add(specific_gene_info)
#     joind_gene = ", ".join(list(gene_set))
#     joind_specific_gene_set = ", ".join(list(specific_gene_set))
#     return [joind_gene, joind_specific_gene_set]

# joined_values_entry = ", ".join(taxonomy_row_list[1])
# # aggragated_taxonomy1 = df_taxonomy.where(df_taxonomy["TAXONOMY"].str.contains(joined_values_entry)).dropna()
# aggragated_taxonomy = df_taxonomy.where(df_taxonomy["TAXONOMY"].str.contains("'Ecdysozoa', 'Arthropoda', 'Hexapoda', 'Insecta', 'Pterygota', 'Palaeoptera', 'Odonata', 'Zygoptera'")).dropna()
# aggragated_taxonomy1 = df_taxonomy.where(df_taxonomy["TAXONOMY"].str.contains("'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Amphibia', 'Gymnophiona'")).dropna()
# aggragated_taxonomy2 = df_taxonomy.where(df_taxonomy["TAXONOMY"].str.contains("'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Actinopterygii', 'Neopterygii', 'Teleostei'")).dropna()
# # 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Archelosauria', 'Archosauria', 'Dinosauria', 'Saurischia', 'Aves', 'Palaeognathae', 'Apterygiformes'

# aggragated_taxonomy = df_taxonomy.where(df_taxonomy["TAXONOMY"].str.contains("'Ecdysozoa', 'Arthropoda', 'Crustacea', 'Multicrustacea'")).dropna()
# get_gene_info(aggragated_taxonomy, "CONTROL_REGION")
# # get_gene_info(aggragated_taxonomy2, "PREVIOUS_GENE")


